# Model debugging

Debug file. Use `model.py` to load the actual model architecture

In [1]:
import torch
import torch.nn as nn
from torchvision import models

In [2]:

class ResnetBasedModel(nn.Module):
    def __init__(self, train_resnet=False, n_diseases=14, n_features=2048):
        """.
        
        params:
          n_features: Number of features from ImageNet model (Resnet-50 in this case)
        """
        super(ResnetBasedModel, self).__init__()
        self.model_ft = models.resnet50(pretrained=True)
        
        if not train_resnet:
            for param in self.model_ft.parameters():
                param.requires_grad = False

        # REVIEW: is this layer ok?
        self.transition = nn.Sequential(
            nn.Conv2d(n_features, 2048, kernel_size=3, padding=1, stride=1, bias=False),
        )
        self.global_pool = nn.Sequential(
            nn.MaxPool2d(32) # 32 is the maximum value of S --> Global Pooling
        )
        self.prediction = nn.Sequential(
            nn.Linear(n_features, n_diseases),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model_ft.conv1(x)
        x = self.model_ft.bn1(x)
        x = self.model_ft.relu(x)
        x = self.model_ft.maxpool(x)

        x = self.model_ft.layer1(x)
        x = self.model_ft.layer2(x)
        x = self.model_ft.layer3(x)
        x = self.model_ft.layer4(x) # n_samples, n_features = 2048, height, width

        print("After Resnet-50: ", x.size())
        
        x = self.transition(x)
        print("After transition: ", x.size())
        
#         # TODO: Calculate bbox from layers
#         prediction_weights = list(self.prediction.parameters())[0] # size: n_diseases, n_features = 2048
#         # x: activations from prev layer # size: n_samples, n_features, height = 16, width = 16
#         # bbox: for each sample, multiply n_features dimensions
#         # --> activations: n_samples, n_diseases, height, width
        activations = None
        
        x = self.global_pool(x)
        print("After pooling: ", x.size())
        x = x.view(x.size(0), -1)
        print("After view: ", x.size())
        x = self.prediction(x) # n_samples, n_diseases
        print("After prediction: ", x.size())
        return x, activations


In [3]:
model = ResnetBasedModel(n_features=2048)

In [4]:
model

ResnetBasedModel(
  (model_ft): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (downsample): Sequential(
          

In [11]:
n_samples = 2
image_channels = 3
image_width = 512
image_height = 512

image = torch.Tensor(n_samples, image_channels, image_height, image_width)
y, activations = model(image)

y.size()

After Resnet-50:  torch.Size([2, 2048, 16, 16])
After transition:  torch.Size([2, 2048, 16, 16])
After pooling:  torch.Size([2, 2048, 1, 1])
After view:  torch.Size([2, 2048])
After prediction:  torch.Size([2, 14])


torch.Size([2, 14])

In [13]:
a = torch.randn(3, 2)
a

tensor([[ 0.7013,  0.3511],
        [ 0.2651,  0.3893],
        [ 0.4031, -1.2247]])

In [14]:
b = a
b

tensor([[ 0.7013,  0.3511],
        [ 0.2651,  0.3893],
        [ 0.4031, -1.2247]])

In [20]:
b = a.view(6)
b

tensor([ 0.7013,  0.3511,  0.2651,  0.3893,  0.4031, -1.2247])

In [21]:
a

tensor([[ 0.7013,  0.3511],
        [ 0.2651,  0.3893],
        [ 0.4031, -1.2247]])

In [31]:
a = torch.Tensor(4, 2048).to("cuda")
b = torch.Tensor(3, 2048).to("cuda")
a.size(), b.size()

(torch.Size([4, 2048]), torch.Size([3, 2048]))

In [35]:
torch.cat([a, b], out=a)
a.size()

torch.Size([10, 2048])

In [37]:
b

tensor([[2.9676e-26, 4.5870e-41, 2.9676e-26,  ..., 1.4187e+13, 4.5441e+30,
         9.6828e+05],
        [3.3949e-09, 1.5434e+25, 3.1554e+03,  ..., 2.6908e+20, 1.8464e+25,
         1.4987e+10],
        [1.2778e+19, 1.2922e+19, 1.7636e+25,  ..., 5.1696e+07, 3.1550e+03,
         3.4468e+27]], device='cuda:0')